<style>
    pre {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
</style>

<div style="display:flex; justify-content:space-around; align-items:center; background-color:#cccccc; padding:5px; border:2px solid #333333;">
    <a href="https://www.um.es/web/estudios/grados/ciencia-ingenieria-datos/" target="_blank">
    <img src="https://www.um.es/documents/1073494/42130150/LogosimboloUMU-positivo.png" alt="UMU" style="height:200px; width:auto;">
    <a href="https://estudios.upct.es/grado/5251/inicio" target="_blank">
    <img src="https://www.upct.es/contenido/universidad/galeria/identidad-2021/logos/logos-upct/marca-upct/marca-principal/horizontal/azul.png" alt="UPCT" style="height:145px; width:auto;">
</div>

# Asignatura: **Deep Learning**

## Titulación: **Grado en Ciencia e Ingeniería de Datos**

## Práctica 2: Redes Convolucionales
### **Sesión 1/3: Introducción a las Redes Convolucionales**

**Autores**: Antonio Martínez Sánchez, Juan Morales Sánchez, José Luís Sancho Gómez y Juan Antonio Botía Blaya

<div style="page-break-before: always;"></div>

### Contenidos
- [Requisitos](#requisitos)
- [El dataset](#dataset)
- [MLP baseline](#mlp)
- [La red convolucional](#cnn)
- [Predicción](#prediccion)
- [Ejercicios](#ejercicios)

### Requisitos
<a class='anchor' id='requisitos'></a>

Se trabajará con notebooks de [Jupyter](https://jupyter.org/install) con código Python empleando como intérprete la última versión de [Miniconda](https://docs.anaconda.com/miniconda/). Se requiere la preinstalación (se recomienda utilizar [pip](https://pypi.org/project/pip/)) de los siguientes paquetes de Python:

- [Numpy](https://pypi.org/project/numpy/) (computación numérica)
- [Scipy](https://pypi.org/project/scipy/) (computación científica)
- [Tensorflow](https://www.tensorflow.org/install/pip?hl=es-419#linux) que incluye a Keras (deep Learning)
- [Scikit-learn](https://pypi.org/project/scikit-learn/) (machine Learning)
- [Scikit-image](https://pypi.org/project/scikit-image/) (Image processing)
- [Matplotlib](https://pypi.org/project/matplotlib/) and [Seaborn](https://pypi.org/project/seaborn/) (visualización de datos)

### El dataset
<a class='anchor' id='dataset'></a>

Los datos a procesar en esta práctica son un conjunto de imágenes sintéticas de criomicroscopía electrónica (acrónimo en inglés, cryo-EM). Cada imagen contiene una proyección 2D de una instancia de un determinado complejo molecular a una orientación aleatoria, específicamente los hay de cinco clases que según su código en la base de datos Protein Data Bank ([PDB](https://www.rcsb.org/)):
1. [3j9i](https://www.rcsb.org/structure/3J9I)
2. [4cr2](https://www.rcsb.org/structure/4CR2)
3. [4v4r](https://www.rcsb.org/structure/4V4R)
4. [4v94](https://www.rcsb.org/structure/4V94)
5. [6utj](https://www.rcsb.org/structure/6UTJ)

Las imágenes tienen un tamaño de 50x50 píxeles y están guardadas en ficheros en formato PNG. El nombre de estos ficheros determina el nombre de la clase, el identificador de clase y el número de partícula de esta clase. El conjunto de datos se encuentra en el directorio *imgs/SNR_high*.

Para poder entrenar el modelo es necesario cargar las imágenes de disco y convertirlas en tensores (arrays n-dimensionales). Las etiquetas de la clase a las que pertenece cada imagen se pueden extraer del nombre de las imágenes. Es probable que al leer las imágenes de disco se vayan leyendo secuencialmente para cada clase, este orden en las imágenes podría distorsionar el entrenamiento, por esto, una vez leídas, se puede aleatorizar su orden gracias a la funcionalidad que ofrece el paquete para machine learning de Python scikit-learn. Finalmente, se tendrá que particionar los datos de entrada en dos conjuntos: entrenamiento y test.

### MLP baseline
<a class='anchor' id='mlp'></a>

Un perceptrón multicapa (acrónimo en inglés, MLP) es una red neuronal de carácter genérico que puede adaptarse para resolver el problema de la clasificación de imágenes.

A modo de punto de partida, *baseline*, implementaremos un MLP que resuelva el problema de clasificación de las imágenes presentadas en el [Dataset](#dataset) según el complejo molecular al que pertenecen. Este MLP tendrá una arquitectura *encoder* con 4 capas ocultas, con un número de neuronas de 512, 256, 128 y 64 respectivamente, con funciones de activación ReLU. La capa de entrada recibirá la información de cada uno de los píxeles de entrada de forma desestructura, o aplanada, esto es, la red tendrá 50x50=2500 variables de entrada. En cuanto a la salida, tendrá una variable de salida por clases que determina la probabilidad de que una entrada pertenezca a una determinada clase. 

### La red convolucional
<a class='anchor' id='cnn'></a>

Las redes convolucionales implementan filtros que se pueden aplicar a señales que varían en función del tiempo y/o espacio como las imágenes y las señales de vídeo. Estos filtros se basan en la operación de convolución y procesan las señales teniendo en cuenta su redundancia espacial y/o temporal de las señales por lo que suelen ser muy eficientes. Al contrario que el MLP, no son generalistas y por si solas no son capaces de resolver algunas tareas. Sin embargo, las primeras redes convolucionales (CNN) se construyeron para resolver el problemas de clasificación de imágenes [LeNet](https://doi.org/10.1109%2F5.726791), es más a partir de [AlexNet](https://doi.org/10.1145%2F3065386) las CNN han demostrado ser la aproximación más efectiva para esta y otras tareas de análisis de imágenes.

En el problema de clasificación, se utilizan diversas capas CNN en forma de encoder para representar la información de la imagen en un espacio menor al determinado por todos los píxeles de imagen. Ten en cuenta que una capas convolucionales suelen tener pocos parámetros. Después, el MLP clasifica la imagen en función a partir del espacio de representación reducido, lo que a su vez permitirá disminuir considerablemente el tamaño del MLP. 

Los paquetes de Python *tensorflow* y *keras* permiten pueden construir redes neuronales de una manera muy simple. En esta práctica se construirá una red neuronal compuesta por un encoder CNN que extraerá una representación comprimida de la imagen y a su salida se añadirá un MLP para predecir las clases de cada imagen.

### Predicción
<a class='anchor' id='prediccion'></a>

Una vez el modelo se ha entrenado, éste se puede utilizar en modo *forward* para procesar cualquier imagen de entrada. Este proceso es particularmente eficiente si se procesan conjuntos de imágenes.

Una vez entrenado un modelo, se suele realizar una evaluación más detallada empleando otros conjuntos de datos. Aquí vamos a reutilizar los del entrenamiento, preferiblemente las imágenes usadas en el conjunto de validación, para evaluar las prestaciones del modelo. 

En el problema de clasificación multiclase, como es el caso, las matrices de confusión permiten un análisis más detallado de las prestaciones del modelo que las métricas globales como la precisión. Específicamente, permite determinar para cada clase el número de errores y que otras clases se han asignado sus instancias.

### Ejercicios
<a class='anchor' id='ejercicios'></a>

**E1:** Carga las imágenes de entrada y sus identificadores de clase correspondientes. Almacénalos en dos tensores con las dimensiones adecuadas para servir de entrada y salida (entrenamiento) respectivamente de la red neuronal construida.

**E2:** Aleatoriza la posición de las imágenes y consecuentemente los identificadores.

**E3:** Particiona los tensores de entrada en dos, asigna el 80% de las imágenes para el entrenamiento y el 20% para la validación.

**E4:** Construye un MLP que realice la clasificación de las imágenes según se especifica en el apartado [MLP baseline](mlp). Utiliza el algoritmo de optimización y la función de pérdidas que creas más oportuna.

**E5:** ¿Cuántos parámetros a entrenar tendría esta red?

**E6:** Entrena el modelo y encuentra la combinación de tamaño de batch y épocas que genere mejores resultados.

**E7:** Muestra la evolución de la función de pérdidas para los conjuntos de datos de entrenamiento y validación a lo largo del proceso de entrenamiento.

**E8:** ¿Qué precisión tiene el modelo?

**E9:** Construye una red neuronal para clasificar las imágenes del dataset que contenga capas convolucionales y un MLP. Utiliza el algoritmo de optimización y la función de pérdidas que creas más oportuna.

**E10:** ¿Cuántos parámetros a entrenar tendría esta red? Compáralos con los que se obtuvieron con el MLP.

**E11:** Entrena el modelo y determina la precisión de la clasificación. Discute los resultados en comparación con los que se obtuvieron con el MLP

**E12:** Muestra la matriz de confusión resultado de procesar el conjunto de datos de test.

**E13:** ¿Qué dos clases presentan mayor confusión mutua? ¿Por qué? Visualiza unas cuantas imágenes con clasificaciones correctas e incorrectas.